# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [69]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [70]:
cities_df = pd.read_csv('../output_data/cities.csv')
#drop null values
cities_df.dropna()
cities_df.head()



,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [71]:
gmaps.configure(api_key=g_key)

In [72]:
locations = cities_df[["Lat", "Lng"]].astype(float)
humidity = cities_df["Humidity"].astype(float)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                dissipating=False, max_intensity=50,
                                point_radius = 1)


# fig.add_layer(heat_layer)
# fig
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [73]:
fig = gmaps.figure()
weather_df = cities_df.loc
#NARROW DOWN THE CITIES to fit weather conditions
narrowed_df = cities_df[(cities_df["Max Temp"] <= 80) & (cities_df["Max Temp"] >= 70) &  (cities_df["Cloudiness"] ==0) &  (cities_df["Wind Speed"] <= 10)].dropna()

narrowed_df





,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [74]:
#creating variable
hotel_df = narrowed_df

#adding "Hotel Name" column
hotel_df["Hotel Name"] = ""

hotel_df




,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,


In [75]:
#Python Apis - Lesson Folder 3: "Ins Google_places"
#set parameters to search for hotels with 5000
params = {
    "keyword": "hotel",
    "radius": 5000,
    "type": "hotel",
    "key": g_key
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


In [76]:
#Reference 06 Python Apis folder 3 - Airport_Ratings
#Use the lat/lng recovered to identify hotels
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
  lat = row["Lat"]
  lng = row["Lng"] 
  city = row["City"]
  
# change location each iteration while leaving original params in place
params["location"] = f"{lat},{lng}"
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


hotels = requests.get(base_url, params=params).json()
results = hotels['results']
results

try:
    hotel_df.append(hotels["results"][1]["name"])
except (KeyError, IndexError):
    print(f"No hotel found")
    hotels.append("")
    
    

[{'business_status': 'OPERATIONAL',
  'geometry': {'location': {'lat': 32.8799738, 'lng': 59.21613479999999},
   'viewport': {'northeast': {'lat': 32.88132187989272,
     'lng': 59.21742852989271},
    'southwest': {'lat': 32.87862222010727, 'lng': 59.21472887010727}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
  'icon_background_color': '#909CE1',
  'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
  'name': 'مهمان پذیر نوید',
  'place_id': 'ChIJLTVncPZnGj8Re9U-HKuHLWY',
  'rating': 3.5,
  'reference': 'ChIJLTVncPZnGj8Re9U-HKuHLWY',
  'scope': 'GOOGLE',
  'types': ['lodging', 'point_of_interest', 'establishment'],
  'user_ratings_total': 4,
  'vicinity': 'V6H8+XFJ, Birjand'},
 {'business_status': 'OPERATIONAL',
  'geometry': {'location': {'lat': 32.8802023, 'lng': 59.216279},
   'viewport': {'northeast': {'lat': 32.88151937989272,
     'lng': 59.21771057989272},
    'southwest': {'lat': 32.878819720

In [77]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [78]:
# Add marker layer ontop of heat map


# Display figure
